# Pytroch autodiff example

In [6]:
import torch

x = torch.tensor(5.0, requires_grad=True)
f = 2 * (x ** 2) + 5
f.backward()
print(x.grad)  # This will print the derivative of f with respect to x


tensor(20.)


# Units lookups
kcal/(g * Å) in Å/(ps²) wolfram says: <br>

418.4 Å/ps^2 (ångströms per picosecond squared) <br>

maybe I should have used a unit package but I havent used one before


In [86]:
from dataclasses import dataclass
from abc import ABC, abstractmethod
import numpy as np
import torch
from einops import rearrange

@dataclass
class Simulation_constants:
    """Holds simulation constants"""
    m: float = 39.9748 # g/mol , mass
    T: float = 300 # K , T
    sigma: float = 3.4 # A, particle size
    eps: float = 0.24 # kcal/mol interactions_strength
    # A box defined by 2 corners (xi,yi), (xf,yf)
    xi: float = 0 # A
    yi: float = 0 # A
    xf: float = 20 # A
    yf: float = 20 # A


class MD_simulation(ABC):
    def __init__(self,sim_cons,pos_file="intial_pos.csv",vel_file="intial_vel.csv"):
        self.positions = torch.from_numpy(np.genfromtxt(pos_file)) # A
        self.positions.requires_grad = True
        self.velocity  = np.genfromtxt(vel_file) # A/ps
        self.constants = sim_cons
        self.total_steps = 0  

        if self.positions.shape != self.velocity.shape:
            raise ValueError("Positions and velocities must have the same shape")
    

    def get_distances_squared(self):   
        # broadcasting points (p) to get z_i - z_j for each coordinate (c), redundant x_j-x_i 
        diff = rearrange(self.positions, 'p c -> p 1 c') - rearrange(self.positions, 'p c -> 1 p c') # A
        return torch.einsum('ijc,ijc->ij', diff, diff) # A^2

    @abstractmethod   
    def get_potential_energy(self):
        pass

    def get_force(self):
        """Calculates the force as the negative gradient of the potential energy."""
        forces, = torch.autograd.grad(self.get_potential_energy(), self.positions, create_graph=True)
        return -forces #kcal/(mol * A)

    @abstractmethod   
    def step(self):
        """Performs a single step of the Velocity Verlet integration algorithm."""
        pass

    def step_reflect(self):
        """Reflects particles that have crossed the boundary."""
        pass



class MD_simulation_lj_vv(MD_simulation):
    """Implements the Lennard-Jones potential and Velocity Verlet integration step"""

    def get_potential_energy(self):
        """Maybe for calculating the force with autodiff"""
        tmp = self.get_distances_squared() + 1e-9 # added eps, avoid dividing by 0, A^2
        tmp = self.constants.sigma**6/tmp**3 # A^6/A^6 
        tmp = 4*self.constants.eps*tmp*(1-tmp) # kcal/mol
        return torch.einsum("ij->", torch.triu(tmp, diagonal=1)) # kcal/mol

    def step(self):
        a = self.get_force()/self.constants.m #kcal/(mol * A * (g/mol)) = kcal/(g *A) = 418.4 A/(ps^2)
        a_unit_factor_kcal_per_gA_to_A_per_ps2 = 418.4
        a *= a_unit_factor_kcal_per_gA_to_A_per_ps2
        print(f"pos= \n{self.positions}, \nvel = \n{self.velocity}, \na = \n{a}")




In [88]:
sim_cons = Simulation_constants()
sim = MD_simulation_lj_vv(sim_cons)
# sim = MD_simulation_lj_vv(sim_cons,pos_file="ipos.csv",vel_file="ivel.csv")
# dis= sim.get_distances_squared()
# print(dis.detach().numpy())
# print(sim.get_potential_energy())
# print(sim.get_potential_energy().backward())
sim.step()
# print(sim.positions)

pos= 
tensor([[18.2100,  6.6900],
        [18.9800, 18.6100],
        [ 2.1100, 13.9000],
        [ 4.8500,  3.5500],
        [ 6.1400, 10.6100],
        [12.0900,  9.9700],
        [11.3300,  3.4800],
        [12.7100, 17.7300],
        [10.2100, 14.2600],
        [ 0.0500,  2.7400],
        [15.6600, 14.7400],
        [ 6.5800, 17.6600],
        [14.7000,  1.2100],
        [ 8.2000,  6.2500],
        [19.0300, 11.2900],
        [ 2.0600,  7.8300]], dtype=torch.float64, requires_grad=True), 
vel = 
[[ 0.26  1.28]
 [-1.55  0.29]
 [-0.72  4.23]
 [ 0.65 -1.69]
 [ 0.86 -2.02]
 [ 1.28 -2.08]
 [-2.05 -2.04]
 [-3.49 -1.95]
 [ 2.56  3.81]
 [ 2.27  2.89]
 [ 3.12  3.02]
 [ 3.3  -0.58]
 [-2.69 -3.48]
 [ 1.05 -3.91]
 [-3.48 -0.97]
 [-1.34  3.22]], 
a = 
tensor([[ 0.0468, -1.2387],
        [ 0.7882,  0.7736],
        [-0.9304,  0.5490],
        [-0.0329, -1.7289],
        [ 0.1917,  0.8033],
        [ 0.9375, -0.9179],
        [ 0.1214, -0.5105],
        [-0.1530,  2.7415],
        [-0.9582, -0.50